In [ ]:
#python3 -m venv elbaff_iesta_venv --prompt="elbaff_iesta_venv"
#source elbaff_iesta_venv/bin/activate

In [ ]:
from nlpaf.annotator.pipeline.pipeline_base import Pipeline
import pandas as pd

import pandas as pd
import numpy as np

import iesta.loader as loader
import iesta.properties as prop  
import iesta.processor as proc  
import iesta.feature_extractor as fe
from iesta.machine_learning.dataloader import IESTAData, METHODOLOGY

from collections import Counter




In [ ]:
conservative_each_dataloader = IESTAData(ideology=prop.CONSERVATIVE_IDEOLOGY, methodology=METHODOLOGY.EACH)





In [ ]:
conservative_training_each_df, conservative_data_path = conservative_each_dataloader.get_training_data()

In [ ]:
conservative_each_dataloader.pivot_df

In [ ]:
conservative_each_dataloader.pivot_binary_effect

In [ ]:
#conservative_training_each_df  = conservative_each_dataloader.data_df[conservative_each_dataloader.data_df["split"] == "training"].copy()
len(conservative_training_each_df)

In [ ]:

conservative_training_each_df.columns.tolist()

In [ ]:
import dataclasses
class TransformerBasedFeaturePipeline(Pipeline):
    

    def __init__(self,
                 input= None,
                 load_default_pipe_configs = True,
                 extended_pipe_configs:dict = None,
                 save_output= False,
                 out_path = None,
                 argument_col:str = "cleaned_text"
                 ):
        super().__init__(input,
                 load_default_pipe_configs,
                 extended_pipe_configs,
                 save_output,
                 out_path)
        self.argument_col= argument_col
        
    def process_input(self) -> list:
        processed = []
        txt_df = self.input[["id", self.argument_col]].copy()
        txt_df =txt_df.rename(columns={"id": "input_id", self.argument_col: "text"},)
        
        for idx, row in txt_df.iterrows():
            processed.append((row.text, {"input_id": row.input_id}))

        return processed
    
    
    def init_and_run(self):
        self.add_annotation_pipe(name = "sentencizer", save_output= False,is_spacy=True, is_native=True)
        self.add_annotation_pipe(name = "EmotionPipeOrchestrator", save_output= True, is_spacy=True)
        #self.add_annotation_pipe(name = "HedgePipeOrchestrator",   save_output= True, is_spacy=True)
        self.add_annotation_pipe(name = "ToxicityOrchestrator",    save_output= True, is_spacy=True)
        self.init_pipe_stack()
        

In [34]:
from nlpaf.util.timer import Timer

import pyarrow.parquet as pq
from pathlib import Path


batch_size = 100
counter = 1



parquet_file = pq.ParquetFile(conservative_data_path)
pipeline = TransformerBasedFeaturePipeline(save_output= True)
pipeline.init_and_run()
out_file = "../data/extracted_features/conservative_batch{}_{}_emotion-toxic.parquet"
for batch in parquet_file.iter_batches(batch_size=batch_size):
    file = out_file.format(counter, batch_size)
    if  Path(file).is_file():
        print(f"skipping batch {counter}_{batch_size}")
        counter = counter+1
        continue
    batch_df = batch.to_pandas()
    
    print(f"processing batch {counter}")
    
    ## RESET
    pipeline.reset_input_output()
    pipeline.out_path = file
    
    pipeline.set_input(batch_df)
    t = Timer(name=f"chunck_{counter}")
    t.start()
    
    pipeline.annotate()
    pipeline.save()
    t.stop()
    #pipeline.out_df.head()
    counter = counter+1


nlpaf        INFO     adding pipe with name EmotionPipeOrchestrator
nlpaf        INFO     orchestrator was initialized successfully
nlpaf        INFO     adding pipe with code emotion_hartmann_component
nlpaf        INFO     adding pipe with name ToxicityOrchestrator
nlpaf        INFO     orchestrator was initialized successfully
nlpaf        INFO     adding pipe with code toxicity_component
nlpaf        INFO     Defining pipe default and spacy stacks
Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification 

skipping batch 1_100
skipping batch 2_100
processing batch 3


  0%|          | 0/3 [00:00<?, ?it/s]nlpaf        INFO     saving annotations of Pipe(name='EmotionPipeOrchestrator', save_output=True, is_spacy=True, is_native=False, pipe_id_or_func='emotion_hartmann_component')
nlpaf        INFO     saving annotations of Pipe(name='ToxicityOrchestrator', save_output=True, is_spacy=True, is_native=False, pipe_id_or_func='toxicity_component')
100%|██████████| 3/3 [00:00<00:00, 61.65it/s]


chunck_3 Elapsed time: 95.3526 seconds
chunck_3 Elapsed time: 95.3526 seconds
processing batch 4


  0%|          | 0/3 [00:00<?, ?it/s]nlpaf        INFO     saving annotations of Pipe(name='EmotionPipeOrchestrator', save_output=True, is_spacy=True, is_native=False, pipe_id_or_func='emotion_hartmann_component')
nlpaf        INFO     saving annotations of Pipe(name='ToxicityOrchestrator', save_output=True, is_spacy=True, is_native=False, pipe_id_or_func='toxicity_component')
100%|██████████| 3/3 [00:00<00:00, 58.69it/s]


chunck_4 Elapsed time: 71.1144 seconds
chunck_4 Elapsed time: 71.1144 seconds
processing batch 5


In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import pipeline

model_path = 'SkolkovoInstitute/roberta_toxicity_classifier'
tokenizer = RobertaTokenizer.from_pretrained(model_path)
model = RobertaForSequenceClassification.from_pretrained(model_path)

_nlp = pipeline('text-classification',model=model,tokenizer=tokenizer,truncation=True, top_k=None)


In [ ]:
import spacy
import nlpaf
#spacy.cli.download("en_core_web_sm")
nlp =spacy.blank("en")
nlp.add_pipe("sentencizer")
nlp.add_pipe("emotion_hartmann_component")
out = list(nlp.pipe(["asda" , "abc"],  n_process=1))

In [ ]:
import pandas  as pd
test_df = pd.DataFrame([{"id": 1, "cleaned_text": "This is a very hard time, I am devastated! This a happy sentence 2."},
                         {"id": 2, "cleaned_text": "You can not get your tiny brain to work on this so stupid!!"},
                         {"id": 3, "cleaned_text": "The amonium Nitrate was sitting there for ages."},
                         {"id": 4, "cleaned_text": "I love you and I love how you look"}
                       ])




In [ ]:
pipeline = TransformerBasedFeaturePipeline(save_output= True,
                 "../data/extracted_features/features_conservative_training_each.parquet")
#pipeline.set_input(conservative_training_each_df)
pipeline.set_input(test_df)
pipeline.init_and_run()
pipeline.out_df.head()